
# Incorporating OLI Calculations with WaterTAP

#### Contact: Paul Vecchiarelli (paul.vecchiarelli@nrel.gov)

This tutorial will demonstrate basic usage of OLI Cloud calls using our custom API tools.

## Rationale

 - Simulations for realistic water sources are mathematically complex: 
 > $ Interactions \ge Cations * Anions$
 - OLI improves WaterTAP approximations and offloads computational resources

## Required OLI API Inputs


 - State variables (solute concentrations, temperature, pressure), which can beextracted from a state block
 
 - Login credentials
 
 - A chemistry (*.dbs) file
     - establishes state variables, phases, etc. to be considered in flash calls

In [ ]:
# used to specify state/convert variables
from pyomo.environ import units as pyunits

# used to build survey lists
from numpy import linspace

# used to execute OLI Cloud functions
from watertap.tools.oli_api.flash import Flash
from watertap.tools.oli_api.credentials import CredentialManager
from watertap.tools.oli_api.client import OLIApi

# 1. Specify State Variables.

- This data is used to construct inputs to OLI Cloud

In [ ]:
    source_water = {
        "temperature": 298.15,
        "pressure": 101325,
        "components": {
            "Cl_-": 870,
            "Na_+": 739,
            "SO4_2-": 1011,
            "Mg_2+": 90,
            "Ca_2+": 258,
            "K_+": 9,
            "HCO3_-": 385,
            "SiO2": 30,
        },
        "units": {
            "temperature": pyunits.K,
            "pressure": pyunits.Pa,
            "components": pyunits.mg / pyunits.L,
        },
    }

# 2. Initialize Flash Instance.

 - We will run most of our methods with this class

In [ ]:
f = Flash()

# 3. Get Survey Parameters.

 - In this example, we will generate a temperature sweep survey

In [ ]:
# each call takes a few seconds
survey_conditions = {
    "Temperature": linspace(273, 373, 6),
    #"SO4_2-": linspace(1e2, 1e3, 10),
    #"Ca_2+": linspace(1e2, 1e3, 10),
}

survey = f.build_survey(
    survey_conditions,
    get_oli_names=True,
)

# 4. Login to OLI Cloud.

- The following code demonstrates an OLI Cloud login:

In [ ]:
# replace dummy credentials with your own, then uncomment CredentialManager instantiation
credentials = {
    "username": "dummy@dummy.com",            
    "password": "dummy",            
    "root_url": "https://dummy.com",            
    "auth_url": "https://dummy.com/dummy", 
}

"""
try:
    credential_manager = CredentialManager(**credentials)
except:
    credential_manager = None
"""

In [ ]:
# after first login, you can use an encryption key to fetch credentials
key = ""

try:
    credential_manager = CredentialManager(encryption_key=key)
except:
    credential_manager = None

# 5. Create *.dbs File and 6. Get Output

In [ ]:
try:
    with OLIApi(credential_manager) as oliapi:
        dbs_file_id = oliapi.get_dbs_file_id(
            chemistry_source=source_water["components"],
            phases=["liquid1", "solid"],
            model_name="tutorial_model",
        )

        water_analysis_base_case = f.build_flash_calculation_input(
            flash_method="wateranalysis",
            state_vars=source_water,                                                       
        )

        water_analysis_composition_survey = f.run_flash(
            flash_method="wateranalysis",
            oliapi_instance=oliapi,
            dbs_file_id=dbs_file_id,
            initial_input=water_analysis_base_case,
            survey=survey,
            file_name="wateranalysis_composition_survey_temperature",
        )
except:
    water_analysis_composition_survey = None

# 7. Extract Filtered Output

 - OLI's output is robust, so WaterTAP enables printing selected results:

In [ ]:
properties = [
    "prescalingTendencies",
    "entropy",
    "gibbsFreeEnergy",
    "selfDiffusivities",
    "molecularConcentration",
    "kValuesMBased",
]

if water_analysis_composition_survey:
    extracted_properties = f.extract_properties(
        water_analysis_composition_survey,
        properties,
        filter_zero=True,
        file_name="properties",
    )